In [1]:
import modeling_functions_script as md

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

In [2]:
# Dataset read
dataset = pd.read_csv('./data.csv' , sep= ';')

In [3]:
# Params lists
pixel = 32 # Image shape reduction to 32 x 32

pool_sizes_1 = [(2,2) , (3,3) , (4,4) , (5,5)]
pool_sizes_2 = [(2,2) , (3,3) , (4,4) , (5,5)]
pool_sizes_3 = [(2,2) , (3,3) , (4,4) , (5,5)]

n_neurons = [400, 500, 600]

epochs = [20 , 30 , 40 , 50]
batchs = [32 , 64 , 128]

# Results lists

model_list = []
loss_list = []
train_accuracy_list = []
val_accuracy_list = []
test_loss_list = []
test_accuracy_list = []


In [4]:
image_list = md.read_images(dataset , pixel)

# Train, validation and test data
X_train , X_val , X_test , y_train , y_val , y_test = md.train_test(dataset , image_list)

for pool_size_1 in pool_sizes_1:
    for pool_size_2 in pool_sizes_2:
        for pool_size_3 in pool_sizes_3:
            for n in n_neurons:
                model = md.model_params(pixel , pool_size_1 , pool_size_2 , pool_size_3 , n)

                for epoch in epochs:
                    for batch in batchs:

                        model_list.append((pixel , pool_size_1 , pool_size_2 , pool_size_3 , n , epoch , batch))
                        history , trained_model = md.model_training(model , epoch , batch , X_train , y_train , X_val , y_val)

                        history_df = pd.DataFrame(history.history)
                        loss_list.append(history_df['loss'][epoch-1])
                        train_accuracy_list.append(history_df['accuracy'][epoch-1])
                        val_accuracy_list.append(history_df['val_accuracy'][epoch-1])
                        test_results = trained_model.evaluate(X_test, y_test)
                        test_loss_list.append(test_results[0])
                        test_accuracy_list.append(test_results[1])

-----------Modelo ==>  32 (2, 2) (2, 2) (2, 2) 400 ---------------
Epoch 1/20
94/94 [==============================] - 2s 21ms/step - loss: 0.8645 - accuracy: 0.5490 - val_loss: 0.6887 - val_accuracy: 0.6383
Epoch 2/20
94/94 [==============================] - 2s 20ms/step - loss: 0.6355 - accuracy: 0.7130 - val_loss: 0.6417 - val_accuracy: 0.7217
Epoch 3/20
94/94 [==============================] - 2s 19ms/step - loss: 0.5707 - accuracy: 0.7463 - val_loss: 0.5356 - val_accuracy: 0.7667
Epoch 4/20
94/94 [==============================] - 2s 20ms/step - loss: 0.5387 - accuracy: 0.7627 - val_loss: 0.4624 - val_accuracy: 0.7983
Epoch 5/20
94/94 [==============================] - 2s 20ms/step - loss: 0.5044 - accuracy: 0.7890 - val_loss: 0.4812 - val_accuracy: 0.7767
Epoch 6/20
94/94 [==============================] - 2s 20ms/step - loss: 0.4857 - accuracy: 0.7867 - val_loss: 0.4531 - val_accuracy: 0.8017
Epoch 7/20
94/94 [==============================] - 2s 20ms/step - loss: 0.4439 - accur

ValueError: Exception encountered when calling layer "max_pooling2d_29" (type MaxPooling2D).

Negative dimension size caused by subtracting 5 from 4 for '{{node max_pooling2d_29/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 5, 5, 1], padding="VALID", strides=[1, 5, 5, 1]](Placeholder)' with input shapes: [?,4,4,128].

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 4, 4, 128), dtype=float32)

In [5]:
model_index = pd.MultiIndex.from_tuples(model_list)
results_array = np.array([loss_list , train_accuracy_list , val_accuracy_list , test_loss_list , test_accuracy_list]).transpose()
results_df = pd.DataFrame(results_array , index=model_index , columns= ['train loss' , 'train accuracy' , 'val accuracy' , 'test loss' , 'test accuracy'])
results_df.to_csv('./modeling_results/modeling_32p_results_df.csv' , sep= ';' , index= False)

In [7]:
results_df.max()

train loss        0.320858
train accuracy    0.996000
val accuracy      0.878333
test loss         0.927828
test accuracy     0.885556
dtype: float64